In [1]:
import onnxruntime as ort
print(ort.get_available_providers())

['CoreMLExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']


In [ ]:
from inference import get_model
import supervision as sv
import cv2
from dotenv import load_dotenv
import os

# Load environment variables and API key
load_dotenv()
ROBOFLOW_API_KEY_ACCOUNT_1 = os.getenv("ROBOFLOW_API_KEY_ACCOUNT_1")
ROBOFLOW_API_KEY_ACCOUNT_2 = os.getenv("ROBOFLOW_API_KEY_ACCOUNT_2")

# Load your trained model from Roboflow
model = get_model(model_id="object-detection-e9zwf/3", api_key=ROBOFLOW_API_KEY_ACCOUNT_2)

# Initialize webcam (0 = default webcam)
cap = cv2.VideoCapture(0)

# Supervision annotators
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Run inference
    results = model.infer(frame)[0]

    # Get the prediction with highest confidence
    if results.predictions:
        highest_confidence_pred = max(results.predictions, key=lambda pred: pred.confidence)
        accuracy = highest_confidence_pred.confidence
    else:
        accuracy = 0.0

    # Convert results to Supervision Detections
    detections = sv.Detections.from_inference(results)
    
    # Annotate frame
    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
    
    # Display accuracy on the frame
    cv2.putText(annotated_frame, f"Accuracy: {accuracy:.2%}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Show frame
    cv2.imshow("Real-time Detection", annotated_frame)
    
    # Break when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.To suppress this warning, set QWEN_2_5_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM2 model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM2_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM3 model. Install SAM3 dependencies and set CORE_MODEL_SAM3_ENABLED to True.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install missing requirements.To suppress this warning, set CORE_MODEL_GAZ

: 